#Imports libs

In [ ]:
import re
import regex
import pandas as pd
import numpy as np
import networkx as nx
import plotly.express as px

#Reading files

In [ ]:
#File path
fileMessages = 'drive/My Drive/1FP6/chat_messages.txt'
fileNames = 'drive/My Drive/common_names.txt'

In [ ]:
common_names = []
with open(fileNames, encoding='UTF-8') as f:
  while True:
    line = f.readline()
    if not line:
      break
    common_names.append(line.split("\n")[0])
common_names[0]

'Michael'

In [ ]:
#This function transform the author name of message in respective int number
def stringToNumber(author):
  author = re.sub('[\.\? ]', '', author.lower())
  number = ''
  for letter in author:
    number = number+str(ord(letter) - 96)
  return int(number)

In [ ]:
#This function indicate whether line of group chat is a new message or is a continuation of previous message
def isNewMessage(line):
  pattern = '^([0-9]+)(\-)([0-9]+)(\-)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
  result = re.match(pattern, line)
  return True if result else False

In [ ]:
#Read lines of the group chat
preData = []
with open(fileMessages, encoding='UTF-8') as f:
  f.readline()
  
  messageBuffer = []
  date, time, author = None, None, None
  while True:
    line = f.readline()
    
    if not line:
      break
    
    if isNewMessage(line) and line.split(' - ')[1].find(':') != -1:
      if len(messageBuffer) > 0 and author.find('55') == -1:
        preData.append([date, time, author, ''.join(messageBuffer)])
      
      messageBuffer.clear()

      lineSplit = line.split(' - ')
      date = lineSplit[0].split(',')[0]
      
      time = lineSplit[0].split(' ')[1]
      
      author = lineSplit[1].split(':')[0]
      
      messageBuffer.append(lineSplit[1].split(':')[1].split('\n')[0])
    else:
      messageBuffer.append(line)


In [ ]:
df = pd.DataFrame(preData, columns=['Date','Time', 'Author', 'Message'])

#Remove line that is not necessary
df.drop(0,inplace=True)

In [ ]:
#Saving quantity of members in group
num_members = len(df.Author.unique())
members_array = df.Author.unique()

In [ ]:
#Removing message that midia file
df = df.loc[~df.Message.str.contains('<Media omitted>')]

#Removing message that contains contact file (.vcf)
df = df[~df.Message.str.contains('.vcf')]

df.reset_index(inplace=True)
df.drop(axis=1,labels='index', inplace=True)

df.Date = pd.to_datetime(df.Date)

In [ ]:
#Dictionary of the members (this dict will be support for the creation adjacency matrix) 
members_dict = {}
for index,member in enumerate(members_array):
  members_dict[member] = index

In [ ]:
members_num_dict = {}
for index,member in enumerate(members_array):
  members_num_dict[index] = member

In [ ]:
#Creating the adjacency matrix
## If the author of the message quotes a member, the adjacency is create
adjacency_matrix = np.zeros((num_members,num_members))

for index in df.index:
  message_split = df.iloc[index].Message.split(' ')
  members_found = [word for word in message_split if word in members_array]
  for member in members_found:
    if adjacency_matrix[ members_dict[ df.iloc[index].Author ] ][ members_dict[member] ] == 0:
      adjacency_matrix[ members_dict[df.iloc[index].Author] ][ members_dict[member] ] = 1

In [ ]:
#For that confidentiality is mainteined
members_anonymized = []
for index,member in enumerate(members_array):
  members_anonymized.append(common_names[stringToNumber(member)%len(common_names)])

#Create dataframe adjancy
df_adjancy = pd.DataFrame(data=adjacency_matrix, index=members_anonymized, columns=members_anonymized)

In [ ]:
df_adjancy

,Kevin,Telesforo,Rita,Norvell,Susane,Betheny,Josselyn,Aya,Tenile,Mckay,Arianne,Latiesha,Claudius,Jolee,Crissie,Keah,Tiffiny,Cain,Kierston,Senaida,Adlai,Roderic,Trecia,Takendra
Kevin,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
Telesforo,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
Rita,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
Norvell,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Susane,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
Betheny,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
Josselyn,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
Aya,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Tenile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mckay,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
#Export the df for the usage in Gephi
df_adjancy.to_csv('matrix_adjacency.csv')

#Metrics

In [ ]:
graph = nx.from_numpy_matrix(adjacency_matrix, create_using=nx.DiGraph())

In [ ]:
graph.number_of_edges()

178

In [ ]:
graph.number_of_nodes()

24

In [ ]:
nx.density(graph)

0.322463768115942

In [ ]:
aux = nx.clustering(nx.Graph(graph))
df_clustering = pd.DataFrame(aux.values())
fig = px.histogram(df_clustering,x=0,)
fig.show()